In [162]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
from datetime import datetime
import pandas as pd
import time

df_joueurs = pd.read_csv('premier_league_joueurs_2024_2025.csv')
df_matchs = pd.read_csv('premier_league_matchs_2024_2025.csv')


# Afficher les valeurs manquantes AVANT



In [163]:
print(df_joueurs.isnull().sum())


equipe            0
joueur            0
nationalite      46
poste            40
age               5
matchs_joues      0
minutes         128
dtype: int64


In [164]:
df_joueurs["minutes"].fillna(0, inplace=True)
df_joueurs["age"].fillna(0, inplace=True)


C:\Users\abirm\AppData\Local\Temp\ipykernel_14588\1359344579.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_joueurs["minutes"].fillna(0, inplace=True)
C:\Users\abirm\AppData\Local\Temp\ipykernel_14588\1359344579.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exampl

In [165]:
df_joueurs.dropna(subset=['poste'], inplace=True)


# Afficher les valeurs manquantes APRES

In [166]:
df_joueurs.isna().sum()

equipe          0
joueur          0
nationalite     6
poste           0
age             0
matchs_joues    0
minutes         0
dtype: int64

In [167]:
df_joueurs[df_joueurs.isna().any(axis=1)]


,equipe,joueur,nationalite,poste,age,matchs_joues,minutes
572,Wolves,Mateus Mane,NaN,MF,0.0,1,2
651,Leicester City,Jeremy Monga,NaN,"FW,MF",15.0,7,112
653,Leicester City,Jake Evans,NaN,FW,0.0,4,24
654,Leicester City,Olabade Aluko,NaN,DF,0.0,1,2
699,Ipswich Town,Tom Taylor,NaN,MF,0.0,0,0
738,Southampton,Jayden Moore,NaN,DF,0.0,0,0


In [168]:
df_joueurs.loc[df_joueurs["joueur"] == "Mateus Mane", "nationalite"] = "Portugaise"
df_joueurs.loc[df_joueurs["joueur"] == "Jeremy Monga", "nationalite"] = "Française"
df_joueurs.loc[df_joueurs["joueur"] == "Jake Evans", "nationalite"] = "Anglaise"
df_joueurs.loc[df_joueurs["joueur"] == "Olabade Aluko", "nationalite"] = "Anglaise"
df_joueurs.loc[df_joueurs["joueur"] == "Tom Taylor", "nationalite"] = "Anglaise"
df_joueurs.loc[df_joueurs["joueur"] == "Jayden Moore", "nationalite"] = "Anglaise"


In [169]:
df_joueurs.isna().sum()

equipe          0
joueur          0
nationalite     0
poste           0
age             0
matchs_joues    0
minutes         0
dtype: int64

In [170]:
df_matchs.isna().sum()

date             0
start_time       0
comp             0
round            0
dayofweek        0
venue            0
result           0
goals_for        0
goals_against    0
opponent         0
xg_for           0
xg_against       0
possession       0
attendance       0
captain          0
formation        0
opp_formation    0
referee          0
team             0
dtype: int64

# Supprimer les doublons


In [171]:

duplicates_before = df_joueurs.duplicated().sum()
print(duplicates_before)
df_joueurs = df_joueurs.drop_duplicates()

0


In [172]:
len(df_matchs)

760

In [173]:
df_matchs = df_matchs[df_matchs['venue'] == 'Home'].copy()


In [174]:
len(df_matchs)


380

# harmoniser 

In [180]:
df_joueurs.columns = df_joueurs.columns.str.lower().str.strip().str.replace(' ', '_')
df_matchs.columns = df_matchs.columns.str.lower().str.strip().str.replace(' ', '_')

In [181]:
df_joueurs.head()

,equipe,joueur,nationalite,poste,age,matchs_joues,minutes
0,Liverpool,Mohamed Salah,eg EGY,FW,32,38,3371
1,Liverpool,Virgil van Dijk,nl NED,DF,33,37,3330
2,Liverpool,Ryan Gravenberch,nl NED,MF,22,37,3160
3,Liverpool,Alexis Mac Allister,ar ARG,MF,25,35,2599
4,Liverpool,Ibrahima Konaté,fr FRA,DF,25,31,2560


# Convertir les types de données


In [176]:
df_joueurs["age"] = df_joueurs["age"].astype(int)
df_joueurs["minutes"] = df_joueurs["minutes"].str.replace(',', '').astype(float).astype('Int64')
df_matchs['date'] = pd.to_datetime(df_matchs['date'], format='%Y-%m-%d')
numeric_cols = ['goals_for', 'goals_against', 'xg_for', 'xg_against', 'possession']
for col in numeric_cols:
    df_matchs[col] = pd.to_numeric(df_matchs[col], errors='coerce')
text_cols_matchs = ['team', 'opponent', 'captain', 'referee', 'comp', 'round', 'venue', 'result', 'dayofweek', 'formation', 'opp_formation']
for col in text_cols_matchs:
    if col in df_matchs.columns:
        df_matchs[col] = df_matchs[col].str.strip()
        
text_cols_joueurs = ['joueur', 'equipe', 'poste', 'nationalite']
for col in text_cols_joueurs:
    if col in df_joueurs.columns:
        df_joueurs[col] = df_joueurs[col].str.strip()
    


In [177]:
df_joueurs.head()


,equipe,joueur,nationalite,poste,age,matchs_joues,minutes
0,Liverpool,Mohamed Salah,eg EGY,FW,32,38,3371
1,Liverpool,Virgil van Dijk,nl NED,DF,33,37,3330
2,Liverpool,Ryan Gravenberch,nl NED,MF,22,37,3160
3,Liverpool,Alexis Mac Allister,ar ARG,MF,25,35,2599
4,Liverpool,Ibrahima Konaté,fr FRA,DF,25,31,2560


In [178]:
df_matchs.head()

,date,start_time,comp,round,dayofweek,venue,result,goals_for,goals_against,opponent,xg_for,xg_against,possession,attendance,captain,formation,opp_formation,referee,team
1,2024-08-25,16:30,Premier League,Matchweek 2,Sun,Home,W,2,0,Brentford,2.5,0.5,62,"60,017",Virgil van Dijk,4-2-3-1,4-4-2,Stuart Attwell,Liverpool
3,2024-09-14,15:00,Premier League,Matchweek 4,Sat,Home,L,0,1,Nott'ham Forest,0.9,0.4,68,"60,344",Virgil van Dijk,4-2-3-1,4-2-3-1,Michael Oliver,Liverpool
4,2024-09-21,15:00,Premier League,Matchweek 5,Sat,Home,W,3,0,Bournemouth,2.0,1.1,58,"60,347",Virgil van Dijk,4-2-3-1,4-2-3-1,Tony Harrington,Liverpool
7,2024-10-20,16:30,Premier League,Matchweek 8,Sun,Home,W,2,1,Chelsea,1.9,1.0,43,"60,277",Virgil van Dijk,4-2-3-1,4-2-3-1,John Brooks,Liverpool
9,2024-11-02,15:00 (16:00),Premier League,Matchweek 10,Sat,Home,W,2,1,Brighton,1.6,1.0,49,"60,331",Virgil van Dijk,4-2-3-1,4-4-2,Tony Harrington,Liverpool
